In [ ]:
import numpy as np
import pandas as pd
import random
import time
import os
from datetime import datetime
from datetime import timedelta
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense,Dropout
from subprocess import run
import json
import pickle
import glob

In [ ]:
os.environ['GGS_DIR'] = '/kaggle/input/gitaggle-settings/gitaggle-settings'
os.environ['USER_NAME'] = 'Dinh Thoai Tran @ randrise.com'
os.environ['USER_EMAIL'] = 'dinhtt@randrise.com'
os.environ['DT_REPO_URL'] = 'https://github.com/dinhtt-randrise/lottery-randrise.git'
os.environ['DEBUG_ON'] = 'no'
USER_NAME = os.environ.get('USER_NAME', 'Dinh Thoai Tran @ randrise.com')
USER_EMAIL = os.environ.get('USER_EMAIL', 'dinhtt@randrise.com')
DEBUG_ON = os.environ.get('DEBUG_ON', 'no') == 'yes'
DT_REPO_URL = os.environ.get('DT_REPO_URL', 'https://github.com/dinhtt-randrise/lottery-randrise.git')


In [ ]:
!git clone https://github.com/dinhtt-randrise/gitaggle.git

In [ ]:
import gitaggle.github as gggh

TODAY_FORCED = None
READY = False
TODAY = '2024.01.01'
BUY_DATE = '2024.01.02'
root_dir = '/kaggle/working/gh'
lottery_company = 'valottery'
lottery_kind = 'powerball'

!mkdir -p $root_dir

folder_name = DT_REPO_URL.split('/')[-1].replace('.git', '')
repo_dir = root_dir + '/' + folder_name

gggh.clone(DT_REPO_URL, root_dir)

if TODAY_FORCED is None:
    files = glob.glob(repo_dir + '/' + lottery_company + '/' + lottery_kind + '/data/' + lottery_kind + '-*.*.*.csv')
    df = pd.DataFrame({'filename': files})
    df = df.sort_values(by=['filename'], ascending=[False])
    for ri in range(len(df)):
        TODAY = df['filename'].iloc[ri].split('/')[-1].replace('.csv', '').replace(lottery_kind + '-', '')
        d1 = datetime.strptime(TODAY, "%Y.%m.%d")
        d2 = d1 + timedelta(minutes=int(+(60 * 24)))
        BUY_DATE = d2.strftime('%Y.%m.%d')        
        DT_FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/data/' + lottery_kind + '-' + TODAY + '.csv'
        PD_FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/pred/d-csv/' + lottery_kind + '-' + BUY_DATE + '.csv'
        PD2_FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/pred/d-txt/' + lottery_kind + '-' + BUY_DATE + '.txt'
        if os.path.exists(DT_FILE) and not os.path.exists(PD_FILE):
            READY = True
            break
else:
    TODAY = TODAY_FORCED
    d1 = datetime.strptime(TODAY, "%Y.%m.%d")
    d2 = d1 + timedelta(minutes=int(+(60 * 24)))
    BUY_DATE = d2.strftime('%Y.%m.%d') 
DT_FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/data/' + lottery_kind + '-' + TODAY + '.csv'
PD_FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/pred/d-csv/' + lottery_kind + '-' + BUY_DATE + '.csv'
PD2_FILE = repo_dir + '/' + lottery_company + '/' + lottery_kind + '/pred/d-txt/' + lottery_kind + '-' + BUY_DATE + '.txt'


In [ ]:
SNO = 1
CNT_SNO = 5
print('=> [Today] ' + TODAY)
print('=> [Buy Date] ' + BUY_DATE)
FILE = '/kaggle/working/raw/data.csv'
BUY_DATE_SET = {
    'N1': '2023.09.11',
    'N2': '2023.09.11',
    'N3': '2023.09.11',
    'N4': '2023.09.11',
    'N5': '2023.09.11',
    'PB': '2023.09.11'
}
PRV_DATE_SET = {
    'N1': '2023.09.10',
    'N2': '2023.09.10',
    'N3': '2023.09.10',
    'N4': '2023.09.10',
    'N5': '2023.09.10',
    'PB': '2023.09.10'
}
PRV_IDX_SET = {
    'N1': 0,
    'N2': 0,
    'N3': 0,
    'N4': 0,
    'N5': 0,
    'PB': 0
}
MAX_PRV_IDX_SET = {
    'N1': 1,
    'N2': 1,
    'N3': 1,
    'N4': 1,
    'N5': 1,
    'PB': 1
}
HEADERS = {
    'N1': '[ N1 ]',
    'N2': '[ N2 ]',
    'N3': '[ N3 ]',
    'N4': '[ N4 ]',
    'N5': '[ N5 ]',
    'PB': '[ PB ]'
}
DT_MIN_DATE = '2022.01.01'
RUN_TIME = 60 * 60 * 11.5
#RUN_TIME = 60 * 5
START_TIME = time.time()

GROUP_NO = 1
GROUP_SIZE = 75
RUN_NO = (GROUP_NO - 1) * GROUP_SIZE + 1
MAX_RUN_NO = GROUP_NO * GROUP_SIZE
MAX_PD = 5
MAX_COST = 100

In [ ]:
wdf = None
if READY:
    os.system('mkdir -p /kaggle/working/raw')
    os.system('mkdir -p /kaggle/working/pred')
    os.system('mkdir -p /kaggle/working/data')

    #d1 = datetime.strptime(datetime.today().strftime('%Y.%m.%d'), "%Y.%m.%d")
    #d1 = d1 + timedelta(minutes=int(-(60 * 4)))
    #d2 = d1 + timedelta(minutes=int(-(60 * 24)))
    #TODAY = d2.strftime('%Y.%m.%d')
    #BUY_DATE = d1.strftime('%Y.%m.%d')
    for k in BUY_DATE_SET:
        BUY_DATE_SET[k] = BUY_DATE
    for k in PRV_DATE_SET:
        PRV_DATE_SET[k] = TODAY
    wdf = pd.read_csv(DT_FILE)
    wdf = wdf[wdf['date'] >= DT_MIN_DATE]
wdf

In [ ]:
def gen_num():
    return random.randint(1, 69)

def gen_num_pb():
    return random.randint(1, 26)


In [ ]:
if READY:
    for snoi in range(CNT_SNO):
        SNO = snoi + 1

        ldf = None
        ldf2 = None
        TWN = 0

        for HOUR in PRV_DATE_SET:
            if time.time() - START_TIME > RUN_TIME:
                break

            BUY_DATE = BUY_DATE_SET[HOUR]

            TWN = 0
            PRV_DATE = PRV_DATE_SET[HOUR]
            PRV_IDX = PRV_IDX_SET[HOUR]
            MAX_PRV_IDX = MAX_PRV_IDX_SET[HOUR]

            df1 = wdf[wdf['hour'] == HOUR]
            df1 = df1.sort_values(by=['date'], ascending=[False])
            df2 = df1[df1['date'] <= PRV_DATE].reset_index()

            while PRV_DATE is not None and PRV_IDX >= 0 and PRV_IDX + 28 < len(df2):    
                if time.time() - START_TIME > RUN_TIME:
                    break
                if (MAX_PRV_IDX > 0 and PRV_IDX >= MAX_PRV_IDX) or (MAX_PRV_IDX < 0):
                    break

                WMC = 0
                date = df2['date'].iloc[PRV_IDX]
                df3 = df1[df1['date'] > date].reset_index()
                TWN = 0
                if len(df3) > 0:
                    TWN = df3['n'].iloc[len(df3) - 1]

                df = df2[df2['date'] <= date]
                DATE = df['date'].iloc[0]
                N = df['n'].iloc[0]
                dt = datetime.strptime(DATE, "%Y.%m.%d")
                CWD = dt.weekday() # Monday is 0 and Sunday is 6
                print(DATE + ' [' + str(CWD) + '] ' + ' (' + HOUR + ') : ' + str(N))
                WNB = str(N)
                WTC = len(df)
                pdf = df

                GROUP_NO = 1
                RUN_NO = (GROUP_NO - 1) * GROUP_SIZE + 1
                MAX_RUN_NO = GROUP_NO * GROUP_SIZE        
                MAX_ROWS = WTC * 16
                LC = MAX_ROWS // 2

                WIN_NUMBERS = []
                WIN_NUM_WK_0 = []
                WIN_NUM_WK_1 = []
                WIN_NUM_WK_2 = []
                WIN_NUM_WK_3 = []
                WIN_NUM_WK_4 = []
                WIN_NUM_WK_5 = []
                WIN_NUM_WK_6 = []
                WIN_NUM_WK_C = []
                NOR = {}
                for i in range(69):
                    NOR[str(i + 1)] = {}
                    for j in range(7):
                        NOR[str(i + 1)][j] = 0
                for ri in range(len(df)):
                    nb = str(df['n'].iloc[ri])
                    if nb not in NOR:
                        NOR[nb] = {}
                        for wd in range(7):
                            NOR[nb][wd] = 0
                    dt = datetime.strptime(df['date'].iloc[ri], "%Y.%m.%d")
                    wd = dt.weekday() 
                    NOR[nb][wd] += 1
                    if nb == WNB:
                        WMC += 1
                    WIN_NUMBERS.append(nb)
                    if ri == 0:
                        continue
                    if ri <= 7 * 1:
                        if wd == 0:
                            WIN_NUM_WK_0.append(nb)
                        if wd == 1:
                            WIN_NUM_WK_1.append(nb)
                        if wd == 2:
                            WIN_NUM_WK_2.append(nb)
                        if wd == 3:
                            WIN_NUM_WK_3.append(nb)
                        if wd == 4:
                            WIN_NUM_WK_4.append(nb)
                        if wd == 5:
                            WIN_NUM_WK_5.append(nb)
                        if wd == 6:
                            WIN_NUM_WK_6.append(nb)
                        if wd == CWD:
                            WIN_NUM_WK_C.append(nb)

                WR = WMC / WTC

                os.system('rm -rf /kaggle/working/data')
                os.system('mkdir -p /kaggle/working/data')

                MATCH = False
                MC = 0
                MC_0 = 0
                MC_1 = 0
                MC_2 = 0
                MC_3 = 0
                MC_4 = 0
                MC_5 = 0
                MC_6 = 0
                MC_C = 0
                TRC = 0
                rows = []
                no = 1
                while time.time() - START_TIME < RUN_TIME:
                    if HOUR == 'PB':
                        n = gen_num_pb()                    
                    else:
                        n = gen_num()

                    key = str(n)

                    m = 0
                    m0 = 0
                    m_0 = 0
                    m_1 = 0
                    m_2 = 0
                    m_3 = 0
                    m_4 = 0
                    m_5 = 0
                    m_6 = 0
                    m_c = 0
                    mo = 0
                    if n == N:
                        MATCH = True
                        m = 1
                        MC += 1

                    if key in WIN_NUM_WK_0:
                        m_0 = 1
                        MC_0 += 1
                    else:
                        m_0 += NOR[str(n)][0] / WTC
                    if key in WIN_NUM_WK_1:
                        m_1 = 1
                        MC_1 += 1
                    else:
                        m_1 += NOR[str(n)][1] / WTC
                    if key in WIN_NUM_WK_2:
                        m_2 = 1
                        MC_2 += 1
                    else:
                        m_2 += NOR[str(n)][2] / WTC
                    if key in WIN_NUM_WK_3:
                        m_3 = 1
                        MC_3 += 1
                    else:
                        m_3 += NOR[str(n)][3] / WTC
                    if key in WIN_NUM_WK_4:
                        m_4 = 1
                        MC_4 += 1
                    else:
                        m_4 += NOR[str(n)][4] / WTC
                    if key in WIN_NUM_WK_5:
                        m_5 = 1
                        MC_5 += 1
                    else:
                        m_5 += NOR[str(n)][5] / WTC
                    if key in WIN_NUM_WK_6:
                        m_6 = 1
                        MC_6 += 1
                    else:
                        m_6 += NOR[str(n)][6] / WTC
                    if key in WIN_NUM_WK_C:
                        m_c = 1
                        MC_C += 1
                    else:
                        m_c += NOR[str(n)][CWD] / WTC

                    if str(n) in WIN_NUMBERS:
                        mo = 1

                    TRC += 1
                    rw = {'no': no, 'run': RUN_NO, 'wr': 0, 'wr_0': 0, 'wr_1': 0, 'wr_2': 0, 'wr_3': 0, 'wr_4': 0, 'wr_5': 0, 'wr_6': 0, 'wr_c': 0, 'n': n, 'm': m, 'mo': mo, 'm_0': m_0, 'm_1': m_1, 'm_2': m_2, 'm_3': m_3, 'm_4': m_4, 'm_5': m_5, 'm_6': m_6, 'm_c': m_c}
                    rows.append(rw)
                    no += 1

                    if time.time() - START_TIME > RUN_TIME:
                        break

                    if len(rows) >= MAX_ROWS:
                        df = pd.DataFrame(rows)
                        df['wr'] = MC / TRC
                        df['wr_0'] = (MC_0 / TRC)
                        df['wr_1'] = (MC_1 / TRC)
                        df['wr_2'] = (MC_2 / TRC)
                        df['wr_3'] = (MC_3 / TRC)
                        df['wr_4'] = (MC_4 / TRC)
                        df['wr_5'] = (MC_5 / TRC)
                        df['wr_6'] = (MC_6 / TRC)
                        df['wr_c'] = (MC_C / TRC)
                        df.to_csv('/kaggle/working/data/powerball-random-' + DATE + '-' + str(RUN_NO) + '.csv', index=False)
                        #print('L: ' + str(RUN_NO) + ' : ' + str(MC) + ' / ' + str(TRC) + ' = ' + str(MC / TRC) + ' // ' + str(WR))

                        RUN_NO += 1
                        no = 1
                        rows = []
                        TRC = 0
                        MC = 0
                        MC_0 = 0
                        MC_1 = 0
                        MC_2 = 0
                        MC_3 = 0
                        MC_4 = 0
                        MC_5 = 0
                        MC_6 = 0
                        MC_C = 0
                        if RUN_NO > MAX_RUN_NO:
                            break

                #if MATCH:
                #    print('Match!')
                #else:
                #    print('Not Match!')

                df = pdf
                NOR = {}
                NORS = {}
                for i in range(69):
                    NOR[str(i + 1)] = {}
                    for j in range(7):
                        NOR[str(i + 1)][j] = 0
                for ri in range(len(df)):
                    nb = str(df['n'].iloc[ri])
                    if nb not in NOR:
                        NOR[nb] = {}
                        for wd in range(7):
                            NOR[nb][wd] = 0
                    dt = datetime.strptime(df['date'].iloc[ri], "%Y.%m.%d")
                    wd = dt.weekday() 
                    NOR[nb][wd] += 1
                for wd in range(7):
                    NORS[wd] = 0
                    for i in range(69):
                        nb = str(i + 1)
                        if nb in NOR:
                            NORS[wd] += NOR[nb][wd]

                GROUP_NO = 1
                CNT = {}
                CNT_0 = {}
                CNT_1 = {}
                CNT_2 = {}
                CNT_3 = {}
                CNT_4 = {}
                CNT_5 = {}
                CNT_6 = {}
                CNT_C = {}
                LST = []
                CNTC = {}
                CNTMX = {}
                CNTMN = {}
                while GROUP_NO <= 7:
                    #if time.time() - START_TIME > RUN_TIME:
                    #    break

                    run_no = (GROUP_NO - 1) * GROUP_SIZE + 1
                    max_run_no = GROUP_NO * GROUP_SIZE

                    while run_no <= max_run_no and run_no <= MAX_RUN_NO:
                        #print('L: ' + str(GROUP_NO) + ' / ' + str(run_no))

                        fn = '/kaggle/working/data/powerball-random-' + DATE + '-' + str(run_no) + '.csv'
                        if not os.path.exists(fn):
                            run_no += 1
                            continue

                        mdf = pd.read_csv(fn)

                        for ri in range(len(mdf)):
                            n = mdf['n'].iloc[ri]
                            wr = mdf['wr'].iloc[ri]
                            wr_0 = mdf['wr_0'].iloc[ri]
                            wr_1 = mdf['wr_1'].iloc[ri]
                            wr_2 = mdf['wr_2'].iloc[ri]
                            wr_3 = mdf['wr_3'].iloc[ri]
                            wr_4 = mdf['wr_4'].iloc[ri]
                            wr_5 = mdf['wr_5'].iloc[ri]
                            wr_6 = mdf['wr_6'].iloc[ri]
                            wr_c = mdf['wr_c'].iloc[ri]
                            m_0 = mdf['m_0'].iloc[ri]
                            m_1 = mdf['m_1'].iloc[ri]
                            m_2 = mdf['m_2'].iloc[ri]
                            m_3 = mdf['m_3'].iloc[ri]
                            m_4 = mdf['m_4'].iloc[ri]
                            m_5 = mdf['m_5'].iloc[ri]
                            m_6 = mdf['m_6'].iloc[ri]
                            m_c = mdf['m_c'].iloc[ri]
                            wr_0 = wr_0 * m_0
                            wr_1 = wr_1 * m_1
                            wr_2 = wr_2 * m_2
                            wr_3 = wr_3 * m_3
                            wr_4 = wr_4 * m_4
                            wr_5 = wr_5 * m_5
                            wr_6 = wr_6 * m_6
                            wr_c = wr_c * m_c
                            nb = str(n)
                            if nb in CNT:
                                CNT[nb] += wr
                                CNTC[nb] += 1
                                CNT_0[nb] += wr_0
                                CNT_1[nb] += wr_1
                                CNT_2[nb] += wr_2
                                CNT_3[nb] += wr_3
                                CNT_4[nb] += wr_4
                                CNT_5[nb] += wr_5
                                CNT_6[nb] += wr_6
                                CNT_C[nb] += wr_c
                                if wr < CNTMN[nb]:
                                    CNTMN[nb] = wr
                                if wr > CNTMX[nb]:
                                    CNTMX[nb] = wr
                            else:
                                CNT[nb] = wr
                                CNTC[nb] = 1
                                CNTMX[nb] = wr
                                CNTMN[nb] = wr
                                CNT_0[nb] = wr_0
                                CNT_1[nb] = wr_1
                                CNT_2[nb] = wr_2
                                CNT_3[nb] = wr_3
                                CNT_4[nb] = wr_4
                                CNT_5[nb] = wr_5
                                CNT_6[nb] = wr_6
                                CNT_C[nb] = wr_c
                            if nb not in LST:
                                LST.append(nb)
                        run_no += 1

                    if run_no > MAX_RUN_NO:
                        break

                    GROUP_NO += 1

                rows = []
                for ri in range(len(LST)):
                    nb = LST[ri]
                    cnt = CNT[nb]
                    cnta = CNT[nb] + CNT_0[nb] + CNT_1[nb] + CNT_2[nb] + CNT_3[nb] + CNT_4[nb] + CNT_5[nb] + CNT_6[nb] + CNT_C[nb] * 0
                    #cnta = cnta * (NOR[nb][0] + NOR[nb][1] + NOR[nb][2] + NOR[nb][3] + NOR[nb][4] + NOR[nb][5] + NOR[nb][6])
                    cnta = cnta * (NOR[nb][CWD] / NORS[CWD]) + CNTC[nb] * (NOR[nb][CWD] / NORS[CWD])
                    rw = {'nb': nb, 'cnt': cnt, 'cnta': cnta}
                    rows.append(rw)
                df = pd.DataFrame(rows)
                df = df.sort_values(by=['cnta'], ascending=[False])
                LST = df['nb'].values

                RW = {'date': DATE, 'hour': HOUR, 'wr': WR, 'wmc': WMC, 'wtc': WTC, 'next_prv_idx': PRV_IDX, 'wn': TWN, 'n': N, 'n1': 0, 'n2': 0, 'n3': 0, 'n4': 0, 'n5': 0, 's1': 0, 's2': 0, 's3': 0, 's4': 0, 's5': 0, 'd1': '_', 'd2': '_', 'd3': '_', 'd4': '_', 'd5': '_', 'r1': 0, 'r2': 0, 'r3': 0, 'r4': 0, 'r5': 0, 'm': 0}
                RW2 = RW.copy()
                m = 0
                for ri in range(len(LST)):
                    if ri >= MAX_PD:
                        break
                    nb = str(LST[ri])
                    n = int(nb)
                    RW['n' + str(ri + 1)] = n
                    if n == TWN:
                        m = 1
                PRV_IDX += 1
                RW['m'] = m
                RW['next_prv_idx'] = PRV_IDX

                TWN = N

                os.system('rm -rf /kaggle/working/data')

                df = pd.DataFrame([RW])
                if ldf is None:
                    ldf = df
                else:
                    ldf = pd.concat([ldf, df])
                ldf = ldf.sort_values(by=['date', 'hour'], ascending=[False, True])
                #ldf.to_csv('data-' + TODAY + '.csv', index=False)

                df = pd.DataFrame([RW2])
                if ldf2 is None:
                    ldf2 = df
                else:
                    ldf2 = pd.concat([ldf2, df])
                ldf2 = ldf2.sort_values(by=['date', 'hour'], ascending=[False, True])
                #ldf2.to_csv('data-' + TODAY + '-2.csv', index=False)

        d1 = datetime.strptime(BUY_DATE, "%Y.%m.%d")
        sd = d1.strftime('%d %B, %Y')
        text = '===== Predictions for ' + sd + ' ======' + "\n"
        pset = {}
        for hour in HEADERS:
            BUY_DATE = BUY_DATE_SET[hour]

            pset[hour + '.p1'] = 0
            pset[hour + '.p2'] = 0
            pset[hour + '.p3'] = 0
            pset[hour + '.p4'] = 0
            pset[hour + '.p5'] = 0
            df = ldf[ldf['hour'] == hour]
            if len(df) > 0:
                text += "\n" + HEADERS[hour] + "\n\n"
                n1 = df['n1'].iloc[0]
                n2 = df['n2'].iloc[0]
                n3 = df['n3'].iloc[0]
                n4 = df['n4'].iloc[0]
                n5 = df['n5'].iloc[0]
                text += '+ N1: ' + str(n1) + "\n"
                text += '+ N2: ' + str(n2) + "\n"
                text += '+ N3: ' + str(n3) + "\n"
                text += '+ N4: ' + str(n4) + "\n"
                text += '+ N5: ' + str(n5) + "\n"
                pset[hour + '.p1'] = n1
                pset[hour + '.p2'] = n2
                pset[hour + '.p3'] = n3
                pset[hour + '.p4'] = n4
                pset[hour + '.p5'] = n5
        print(text)

        prows = []
        d1 = datetime.strptime(BUY_DATE, "%Y.%m.%d")
        sd = d1.strftime('%d %B, %Y')
        text = '===== Matchable Predictions for ' + sd + ' ======' + "\n"
        for hour in HEADERS:
            BUY_DATE = BUY_DATE_SET[hour]

            df = ldf2[ldf2['hour'] == hour]
            matched = False
            if len(df) > 0:
                mt = 0
                text += "\n" + HEADERS[hour] + "\n\n"
                date = df['date'].iloc[0]
                n1 = df['n1'].iloc[0]
                n2 = df['n2'].iloc[0]
                n3 = df['n3'].iloc[0]
                n4 = df['n4'].iloc[0]
                n5 = df['n5'].iloc[0]
                s1 = df['s1'].iloc[0]
                s2 = df['s2'].iloc[0]
                s3 = df['s3'].iloc[0]
                s4 = df['s4'].iloc[0]
                s5 = df['s5'].iloc[0]
                d1 = df['d1'].iloc[0]
                d2 = df['d2'].iloc[0]
                d3 = df['d3'].iloc[0]
                d4 = df['d4'].iloc[0]
                d5 = df['d5'].iloc[0]
                r1 = df['r1'].iloc[0]
                r2 = df['r2'].iloc[0]
                r3 = df['r3'].iloc[0]
                r4 = df['r4'].iloc[0]
                r5 = df['r5'].iloc[0]
                sa = 0
                if n1 > 0:
                    if s1 > sa:
                        sa = s1
                    mt = 1
                    matched = True
                    text += '+ MN1: ' + str(n1) + ' (Accuracy: ' + str(s1) + ', Last Updated: ' + str(d1) + ', Win Rate in Data: ' + str(r1) + ') ' + "\n"
                if n2 > 0:
                    if s2 > sa:
                        sa = s2
                    mt = 1
                    matched = True
                    text += '+ MN2: ' + str(n2) + ' (Accuracy: ' + str(s2) + ', Last Updated: ' + str(d2) + ', Win Rate in Data: ' + str(r2) + ') ' + "\n"
                if n3 > 0:
                    if s3 > sa:
                        sa = s3
                    mt = 1
                    matched = True
                    text += '+ MN3: ' + str(n3) + ' (Accuracy: ' + str(s3) + ', Last Updated: ' + str(d3) + ', Win Rate in Data: ' + str(r3) + ') ' + "\n"
                if n4 > 0:
                    if s4 > sa:
                        sa = s4
                    mt = 1
                    matched = True
                    text += '+ MN4: ' + str(n4) + ' (Accuracy: ' + str(s4) + ', Last Updated: ' + str(d4) + ', Win Rate in Data: ' + str(r4) + ') ' + "\n"
                if n5 > 0:
                    if s5 > sa:
                        sa = s5
                    mt = 1
                    matched = True
                    text += '+ MN5: ' + str(n5) + ' (Accuracy: ' + str(s5) + ', Last Updated: ' + str(d5) + ', Win Rate in Data: ' + str(r5) + ') ' + "\n"
                if not matched:
                    text += '+ No matchable numbers' + "\n"
                if mt == 1:
                    if sa < 1:
                        mt = 0
                P1 = pset[hour + '.p1']
                P2 = pset[hour + '.p2']
                P3 = pset[hour + '.p3']
                P4 = pset[hour + '.p4']
                P5 = pset[hour + '.p5']
                rw = {'date': date, 'buy_date': BUY_DATE, 'hour': hour, 'sno': SNO, 'mt': mt, 'n1': n1, 'n2': n2, 'n3': n3, 'n4': n4, 'n5': n5, 'p1': P1, 'p2': P2, 'p3': P3, 'p4': P4, 'p5': P5, 's1': s1, 's2': s2, 's3': s3, 's4': s4, 's5': s5, 'd1': d1, 'd2': d2, 'd3': d3, 'd4': d4, 'd5': d5, 'r1': r1, 'r2': r2, 'r3': r3, 'r4': r4, 'r5': r5}
                prows.append(rw)
        print(text)

        df = pd.DataFrame(prows)
        df = df.sort_values(by=['buy_date', 'hour'], ascending=[False, True])
        df.to_csv('pred/powerball-pred-' + str(SNO) + '.csv', index=False)


In [ ]:
ddf = None
if READY:
    for idx in range(CNT_SNO):
        no = idx + 1
        fn = '/kaggle/working/pred/powerball-pred-' + str(no) + '.csv'
        if not os.path.exists(fn):
            continue
        df = pd.read_csv(fn)
        if len(df) == 0:
            continue
        if ddf is None:
            ddf = df
        else:
            ddf = pd.concat([ddf, df])

In [ ]:
if READY:
    frows = []
    ddf = ddf.sort_values(by=['buy_date', 'hour', 'sno'], ascending=[False, True, True])
    #ddf.to_csv('data-' + TODAY + '.csv', index=False)
    dates = ddf['buy_date'].unique()
    text = ''
    for buy_date in dates:
        df2 = ddf[ddf['buy_date'] == buy_date]
        d1 = datetime.strptime(buy_date, "%Y.%m.%d")
        sd = d1.strftime('%d %B, %Y')
        text += "\n" + '===== VALottery: PowerBall @ ' + sd + ' ======' + "\n"
        hours = df2['hour'].unique()
        for hour in hours:
            df3 = df2[df2['hour'] == hour]
            if len(df3) > 0:
                text += "\n" + HEADERS[hour] + "\n\n"

                nlst = []
                plst = []
                rows = []
                for ri in range(len(df3)):
                    for i in range(5):
                        no = i + 1
                        p = df3['p' + str(no)].iloc[ri]
                        if (len(plst) < 10) and (p not in plst):
                            plst.append(p)
                    for i in range(5):
                        no = i + 1
                        n = df3['n' + str(no)].iloc[ri]
                        if n > 0:
                            s = df3['s' + str(no)].iloc[ri]
                            d = df3['d' + str(no)].iloc[ri]
                            r = df3['r' + str(no)].iloc[ri]
                            v = s * r
                            rw = {'n': n, 'v': v, 's': s, 'd': d, 'r': r}
                            rows.append(rw)

                RW = {'date': df3['date'].iloc[0], 'buy_date': buy_date, 'hour': hour, 'p1': 0, 'p2': 0, 'p3': 0, 'p4': 0, 'p5': 0, 'p6': 0, 'p7': 0, 'p8': 0, 'p9': 0, 'p10': 0}
                mt = 0
                if len(rows) > 0:
                    mt = 1
                    df4 = pd.DataFrame(rows)
                    df4 = df4.sort_values(by=['v'], ascending=[False])
                    no = 1
                    for ri in range(len(df4)):
                        n = df4['n'].iloc[ri]
                        if n in nlst:
                            continue
                        nlst.append(n)
                        s = df4['s'].iloc[ri]
                        d = df4['d'].iloc[ri]
                        r = df4['r'].iloc[ri]
                        text += '+ N' + str(no) + ': ' + str(n) + '    (Accuracy: ' + str(s) + ', Win Rate in Data: ' + str(r) + ', Model Last Updated: ' + str(d) + ')' + "\n"
                        no += 1

                        #if no <= 5:
                        #    RW['n' + str(no)] = n
                        #    RW['s' + str(no)] = s
                        #    RW['d' + str(no)] = d
                        #    RW['r' + str(no)] = r
                    #RW['mt'] = mt
                #else:
                #    text += '+ No matchable numbers' + "\n"

                #text += "\n--------------------\n"
                for ri in range(len(plst)):
                    no = ri + 1
                    p = plst[ri]
                    RW['p' + str(no)] = p
                    text += '+ P' + str(no) + ': ' + str(p) + "\n"
                frows.append(RW)

    odf = pd.DataFrame(frows)
    odf = odf.sort_values(by=['buy_date', 'hour'], ascending=[False, True])
    odf.to_csv('powerball-' + BUY_DATE + '.csv', index=False)
    odf.to_csv(PD_FILE, index=False)

In [ ]:
if READY:
    fh = open('/kaggle/working/powerball-' + BUY_DATE + '.txt', 'w')
    fh.write(text)
    fh.close()
    print(text)
    
    fh = open(PD2_FILE, 'w')
    fh.write(text)
    fh.close()

In [ ]:
if READY:
    os.system('rm -rf /kaggle/working/raw')
    os.system('rm -rf /kaggle/working/pred')
    os.system('rm -rf /kaggle/working/data')
    
    gggh.write_text_file('/kaggle/working/msg.txt', 'Predict draw for ' + BUY_DATE)
    gggh.identity(repo_dir, USER_NAME, USER_EMAIL)
    gggh.commit(repo_dir, '/kaggle/working/msg.txt')
    !rm -rf $root_dir
    !rm -rf /kaggle/working/msg.txt

    os.system('rm -rf /kaggle/working/gitaggle')